In [1]:
import pandas as pd
import sqlalchemy as sal
from dotenv import load_dotenv
import numpy as np
import os
load_dotenv()

# Create a connection to the database
server_database = os.getenv("SERVER_DATABASE")
engine = sal.create_engine(f'mssql+pyodbc://@{server_database}?trusted_connection=yes&driver=SQL+Server')
conn = engine.connect()

C:\Users\krittaphas.chi\AppData\Local\Temp\ipykernel_4268\4283691341.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('..\sql\inclusion\icd.sql') as file:
    sql_command = file.read()
    pt_diag = pd.read_sql(sql_command, conn)
    file.close()

print("Number of patients included by OMOP concept IDs:",len(pt_diag))

Number of patients included by OMOP concept IDs: 298427


In [3]:
with open('..\sql\inclusion\drug.sql') as file:
    sql_command = file.read()
    pt_drug = pd.read_sql(sql_command, conn)
    file.close()
print("Number of patients included by lab criteria:",len(pt_drug))

Number of patients included by lab criteria: 249020


In [4]:
print("Number of patients included by code criteria:",len(set(pt_diag['person_id'].values) - set(pt_drug['person_id'].values)))
print("Number of patients included by drug criteria:",len(set(pt_drug['person_id'].values) - set(pt_diag['person_id'].values)))

print("Number of patients satisfied both criteria:",len(pt_diag.merge(pt_drug, on='person_id', how='inner')))



Number of patients included by code criteria: 102727
Number of patients included by drug criteria: 53320
Number of patients satisfied both criteria: 195700


In [5]:
merge_data = pt_diag.merge(pt_drug, on='person_id', how='outer')

In [6]:
merge_data['diag'] = [1 if ~np.isnan(merge_data.loc[i ,'condition_concept_id']) else 0 for i in range(len(merge_data))]
merge_data['drug'] = [1 if ~np.isnan(merge_data.loc[i ,'age_at_first_drug']) else 0 for i in range(len(merge_data))]


In [7]:
merge_data['criteria'] = ['diag' if merge_data.loc[i, 'diag'] == 1 else '' for i in range(len(merge_data))]
merge_data['criteria'] = [x + ' drug' if merge_data.loc[i, 'drug'] == 1 else x for i, x in enumerate(merge_data['criteria'])]

In [8]:
merge_data['criteria'] = merge_data['criteria'].apply(lambda x: x.strip())
merge_data['criteria'] = merge_data['criteria'].apply(lambda x: x.replace(' ', '+'))

In [9]:
merge_data.to_csv('../data/include_pt_28apr24.csv', index=False)

In [10]:
merge_data['criteria'].value_counts()

criteria
diag+drug    195700
diag         102727
drug          53320
Name: count, dtype: int64